In [1]:
## Speed dating algorithm:
# Groups of 6 people, then rotate groups, 
# no 2 people from same lab (ideally), no 2 people meet again
# 10 times

#number of people per group
peopleInGroup = 6

numberOfTimes = 6

In [2]:
# Read excel/csv
import pandas as pd

fileName = 'ListOfPeoplewithPI.xlsx'
listOfPeople = pd.read_excel(fileName)

print(listOfPeople)


           Name    Groups
0       Bejamin     Acton
1        Foakes     Acton
2    Gari Hupta     Acton
3      Heeringa     Acton
4      Horsnell     Acton
..          ...       ...
97        Marsh     Marsh
98     Michaels  Michaels
99      Pichaud   Pichaud
100     Pitcher   Pitcher
101      Stefan    Stefan

[102 rows x 2 columns]


In [3]:
max(listOfPeople.index) + 1

102

In [4]:
# https://stackoverflow.com/questions/36429507/python-combinations-without-repetitions
# Author: hahho
from itertools import chain, repeat, count, islice
from collections import Counter


def repeat_chain(values, counts):
    return chain.from_iterable(map(repeat, values, counts))


def unique_combinations_from_value_counts(values, counts, r):
    n = len(counts)
    indices = list(islice(repeat_chain(count(), counts), r))
    if len(indices) < r:
        return
    while True:
        yield tuple(values[i] for i in indices)
        for i, j in zip(reversed(range(r)), repeat_chain(reversed(range(n)), reversed(counts))):
            if indices[i] != j:
                break
        else:
            return
        j = indices[i] + 1
        for i, j in zip(range(i, r), repeat_chain(count(j), counts[j:])):
            indices[i] = j


def unique_combinations(iterable, r):
    values, counts = zip(*Counter(iterable).items())
    return unique_combinations_from_value_counts(values, counts, r)

In [5]:
# Create list of available people for each person
availableChoices = dict();
for numPerson in listOfPeople.index:
    availableChoices[numPerson] = listOfPeople.index[listOfPeople.Groups[numPerson] != listOfPeople.Groups]

In [6]:
availableChoices[1]

Int64Index([  8,   9,  10,  11,  12,  13,  14,  15,  16,  17,  18,  19,  20,
             21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,
             34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,
             47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,
             60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,
             73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  86,
             87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99,
            100, 101],
           dtype='int64')

In [7]:
import random
import numpy as np
import math



In [ ]:
uniquePeopleMetBest = 0;
repeatedPeopleTotalBest = 100;
groupsPerTimeBest = list();
while uniquePeopleMetBest <= 60 and repeatedPeopleTotalBest >= 1:
    availableChoicesPrev = availableChoices.copy();
    availableChoicesCurrent = availableChoices.copy();

    remainingPeople = list(listOfPeople.index);
    groupsPerTime = list();
    numTime = 0
    while len(groupsPerTime)+1 < ((max(listOfPeople.index) + 1) / peopleInGroup)*numberOfTimes:
        #print(numTime)
        remainingPeople = list(listOfPeople.index);
        try:
            while len(remainingPeople) > 0:
                random_num = random.choice(remainingPeople)
                #print(random_num)
                currentGroup = [random_num];
                remainingPeople.remove(random_num);

                while len(currentGroup) < peopleInGroup:
                    currentAvailableGroup = availableChoicesCurrent[currentGroup[0]]
                    for person in currentGroup:
                        currentAvailableGroup = currentAvailableGroup.intersection(availableChoicesCurrent[person])
                    currentAvailableGroup = currentAvailableGroup.intersection(remainingPeople)
                    random_num = random.choice(remainingPeople)
                    #print(random_num)
                    currentGroup.append(random_num);
                    remainingPeople.remove(random_num);

                for person in currentGroup:
                    for otherPerson in currentGroup:
                        availableChoicesCurrent[person] = availableChoicesCurrent[person].delete(availableChoicesCurrent[person] == otherPerson)

                groupsPerTime.append(currentGroup);
            numTime = numTime + 1;
        except Exception:
            numTime = 0;
            availableChoicesCurrent = availableChoices.copy();
            groupsPerTime = list();

    # Calculate average number of people that have seen each other more than once
    repeatedPeopleTotal = 0;
    uniquePeopleMet = 0;
    for idPerson in listOfPeople.index:
        unique_numbers = list();
        for group in groupsPerTime:
            if idPerson in group:
                for item in group:
                    unique_numbers.append(item)
        repeatedPeopleTotal += len(unique_numbers) - len(set(unique_numbers))
        uniquePeopleMet += len(set(unique_numbers))

    uniquePeopleMetCurrent = uniquePeopleMet / (max(listOfPeople.index) + 1)
    repeatedPeopleTotalCurrent = repeatedPeopleTotal / (max(listOfPeople.index) + 1)
        
    if uniquePeopleMetCurrent > uniquePeopleMetBest and repeatedPeopleTotalCurrent < repeatedPeopleTotalBest:
        print('New Best!')
        groupsPerTimeBest = groupsPerTime.copy();
        uniquePeopleMetBest = uniquePeopleMetCurrent;
        repeatedPeopleTotalBest = repeatedPeopleTotalCurrent;
        print(uniquePeopleMetBest)
        print(repeatedPeopleTotalBest)

New Best!
27.607843137254903
8.392156862745098
New Best!
27.941176470588236
8.058823529411764
New Best!
28.274509803921568
7.7254901960784315
New Best!
28.392156862745097
7.607843137254902


In [ ]:
 ((max(listOfPeople.index) + 1) / peopleInGroup)*numberOfTimes

In [ ]:
len(groupsPerTime)

In [ ]:
unique_numbers